# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Populating tables in Apache Cassandra and performing queries 

## Processing CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Connecting to Apache Cassandra 

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Creating queries to ask the following three questions of the data:

### 1.Querying the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
## PRIMARY KEY description: It consists of two columns, session_id and session_item. 
## These columns are used in the query and also indicate a unique record. 
## session_id is the PARTITION KEY (instead of session_item) because we would like to have the records from a particular session in the same node.
## Moreover, session_item always relates to a particular session so if we want to filter by session_item, we will always specify the session_id as well.

drop_query1 = "DROP TABLE IF EXISTS session_item_info"
create_query1 = """CREATE TABLE IF NOT EXISTS session_item_info (
                    session_id INT, 
                    session_item INT, 
                    artist_name TEXT, 
                    song_title TEXT, 
                    duration DECIMAL, 
                    PRIMARY KEY (session_id, session_item))"""

try:
    session.execute(drop_query1)
    session.execute(create_query1)
except Exception as e:
    print(e)           

In [9]:
# Inserting data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item_info (session_id, session_item, artist_name, song_title, duration) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Performning SELECT statement to verify the data was entered into the table
query = "SELECT artist_name, song_title, duration FROM session_item_info WHERE session_id = 338 AND session_item = 4"
try:
    results = pd.DataFrame(list(session.execute(query)))
    print (results)
except Exception as e:
    print(e)


  artist_name                       song_title  duration
0   Faithless  Music Matters (Mark Knight Dub)  495.3073


### 2. Querying the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [17]:
## PRIMARY KEY description: It consists of user_id, session_id and session_item columns. 
## First two are used in the WHERE condition and session_item is used to indicate unique record as well as is used in sorting.
## Session ID is the PARTITION KEY because we would like to have the records for a particular user in the same node, also the session always relates to a user

drop_query2 = "DROP TABLE IF EXISTS user_session_info"
create_query2 = """CREATE TABLE IF NOT EXISTS user_session_info (
                    user_id INT, 
                    session_id INT, 
                    session_item INT, 
                    artist_name TEXT, 
                    song_title TEXT, 
                    user_first_name TEXT, 
                    user_last_name TEXT, 
                    PRIMARY KEY ((user_id, session_id), session_item))"""

try:
    session.execute(drop_query2)
    session.execute(create_query2)
except Exception as e:
    print(e)     


In [18]:
# Inserting data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_info (user_id, session_id, session_item, artist_name, song_title, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [24]:
## Performning SELECT statement to verify the data was entered into the table
query = "SELECT artist_name, song_title, user_first_name, user_last_name FROM user_session_info WHERE user_id = 10 AND session_id = 182 ORDER BY session_item"
try:
    results = pd.DataFrame(list(session.execute(query)))
    print (results)
except Exception as e:
    print(e)

         artist_name                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  user_first_name user_last_name  
0          Sylvie           Cruz  
1          Sylvie           Cruz  
2          Sylvie           Cruz  
3          Sylvie           Cruz  


### 3. Querying every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [27]:
## PRIMARY KEY description: It consists of song_title, artist_name and user_id columns. 
## song_title is the PARTITION KEY because we use it in the WHERE condition
## Other two are used to indicate unique record. 
## Particular song title may have multiple artists and combination of first and last name may not be unique.

drop_query3 = "DROP TABLE IF EXISTS song_listeners"
create_query3 = """CREATE TABLE IF NOT EXISTS song_listeners (
                    song_title TEXT, 
                    artist_name TEXT, 
                    user_id INT,
                    user_first_name TEXT, 
                    user_last_name TEXT, 
                    PRIMARY KEY (song_title, artist_name, user_id))"""

try:
    session.execute(drop_query3)
    session.execute(create_query3)
except Exception as e:
    print(e)    

In [28]:
# Inserting data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (song_title, artist_name, user_id, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[9], line[0], int(line[10]), line[1], line[4]))

In [29]:
## Performning SELECT statement to verify the data was entered into the table
query = "SELECT user_first_name, user_last_name FROM song_listeners WHERE song_title = 'All Hands Against His Own'"
try:
    results = pd.DataFrame(list(session.execute(query)))
    print (results)
except Exception as e:
    print(e)

  user_first_name user_last_name
0      Jacqueline          Lynch
1           Tegan         Levine
2            Sara        Johnson


### Droping the tables before closing out the sessions

In [30]:
## Dropping tables before closing out the session
try:
    session.execute(drop_query1)
    session.execute(drop_query2)
    session.execute(drop_query3)
except Exception as e:
    print(e)   

### Closing the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()